### Imports all the village codes into csv

In [156]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [124]:
divisions = ["amravati", "aurangabad", "kokan", "nagpur", "nashik", "pune"]
base_url = "http://115.124.110.196:8080/epeek"

In [144]:
district_set = []
for division in divisions:
    district_set += get_all_districts(division)



In [147]:
taluka_set = []

for district in district_set:
    taluka_set += get_all_taluka(district)

In [143]:
def get_all_districts(division):
    url = f"{base_url}/fillDistrict.jsp?div={division}"
    district_response = requests.get(url)
    html_content = bs(district_response.content, "html.parser")
    values  = html_content.findAll("option")
    values.pop(0) # remove the first values for every line
    districts = []
    for value in values:
        row = {}
        row["division"] = division
        row['district_code'] = value["value"]
        row['district'] = value.text
        districts.append(row)
    
    return districts

In [141]:
def get_all_taluka(district):
    url = f"{base_url}/fillTaluka.jsp?dist={district['district_code']}&div={district['division']}"
    taluka_response = requests.get(url)
    html_content = bs(taluka_response.content, "html.parser")
    values  = html_content.findAll("option")
    values.pop(0) # remove the first values for every line
    taluka = []
    for value in values:
        row = {}
        row["division"] = district['division']
        row["district"] = district["district"]
        row['district_code'] = district["district_code"]
        row['taluka_code'] = value["value"]
        row['taluka'] = value.text
        taluka.append(row)
    
    return taluka

In [154]:
village_set = []
for taluka in taluka_set:
    village_set += get_all_village(taluka)

In [153]:
def get_all_village(taluka):
    url = f"{base_url}/fillVillage.jsp?dist={taluka['district_code']}&blk={taluka['taluka_code']}&div={taluka['division']}"
    village_response = requests.get(url)
    html_content = bs(village_response.content, 'html.parser')
    values =  html_content.findAll("option")
    values.pop(0) #remove the first element (for all)
    village = []
    for value in values:
        row = {}
        row["division"] = taluka['division']
        row["district"] = taluka["district"]
        row['district_code'] = taluka["district_code"]
        row['taluka_code'] = taluka["taluka_code"]
        row['taluka'] = taluka["taluka"]
        row["village_code"] = value["value"]
        row["village"] = value.text
        village.append(row)
    
    return village

In [157]:
village_data_set = pd.DataFrame(village_set)

In [158]:
village_data_set

,division,district,district_code,taluka_code,taluka,village_code,village
0,amravati,अमरावती,7,4,अचलपुर,270700040068810000,अजितपुर
1,amravati,अमरावती,7,4,अचलपुर,270700040070040000,असदपुर
2,amravati,अमरावती,7,4,अचलपुर,270700040070290000,अंतरगाव
3,amravati,अमरावती,7,4,अचलपुर,270700040068880000,अंबाडा कंडारी
4,amravati,अमरावती,7,4,अचलपुर,270700040070280000,अंबुज
...,...,...,...,...,...,...,...
44304,pune,कोल्हापूर,34,3,हातकलंगले,273400030420760000,हातकणंगले
44305,pune,कोल्हापूर,34,3,हातकलंगले,273400030420820000,हालोंडी
44306,pune,कोल्हापूर,34,3,हातकलंगले,273400030420710000,हिंगणगाव
44307,pune,कोल्हापूर,34,3,हातकलंगले,273400034340700000,हुपरी
